In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# import matplotlib as mpl
# mpl.rcParams['figure.dpi'] = 150
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk import download, wordnet
nltk.download('omw-1.4')
download('punkt')
download('stopwords')
download("wordnet")
#from nltk.corpus import stopwords
import gensim
from gensim.parsing.preprocessing import remove_stopwords

lemma = wordnet.WordNetLemmatizer()

# For stemming
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

from tqdm import tqdm

#import CountVectorizer if we just want a Bag of Words Model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

# import some classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold, train_test_split as tts

from sklearn.metrics import accuracy_score, confusion_matrix as CM

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
wine_data = pd.read_csv('https://github.com/dvasiliu/AML/blob/main/Data%20Sets/winemagdata130kv2.csv?raw=true',quoting=2)

In [ ]:
#subset 10000 random reviews
data = wine_data.sample(10000,random_state=1693)

In [ ]:
wine_data

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0.0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87.0,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1.0,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87.0,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2.0,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3.0,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4.0,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966.0,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90.0,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967.0,US,Citation is given as much as a decade of bottl...,NaN,90.0,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968.0,France,Well-drained gravel soil gives this wine its c...,Kritt,90.0,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969.0,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90.0,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


---

### A different way of sampling (optional)

In [ ]:
reviews = wine_data.description
ratings = wine_data.points
# get some random indexes
np.random.seed(1693)
indx = np.random.choice(len(ratings),size=10000)

In [ ]:
subset_reviews =reviews[indx]
subset_ratings = ratings[indx]

In [ ]:
wine_data = wine_data.loc[indx]

---

In [ ]:
wine_subset = data[['description','points']].reset_index()

In [ ]:
wine_subset

,index,description,points
0,41414,Of all the Atauta single-vineyard wines from t...,94.0
1,10176,Pleasant red fruits awash in chocolaty oak cra...,87.0
2,123359,This medium-bodied Sauvignon Blanc adds a bit ...,89.0
3,101435,"Aromatically unusual, this suggests raspberry ...",83.0
4,66261,With some tannins and a good balance between a...,88.0
...,...,...,...
9995,110350,"This estate, now owned by the negociant Champy...",90.0
9996,82500,"This is a clean and bright selection, with pur...",87.0
9997,95939,"This is a tight, tangy wine that has been roun...",87.0
9998,64438,"Bright on the nose with purple fruit, black ra...",90.0


---
## Preprocessing of the Text

In [ ]:
# this is the trick: we included a fakeword at the end of each row for the reviews
reviews = wine_subset.description.values + ' '+'fakeword'+' '

In [ ]:
joined_reviews = ' '

for i in tqdm(range(len(reviews))):
  joined_reviews = joined_reviews+reviews[i]

100%|██████████| 10000/10000 [00:00<00:00, 882602.58it/s]


In [ ]:
reviews

array(['Of all the Atauta single-vineyard wines from the sensational 2011 vintage, La Mala is arguably the most overtly oaky and modern in style. Grainy, resiny barrel-driven aromas come with controlled burn and heat. This is tannic and firm as cement, with ample oak resin creating a creamy feel. Resiny, fully spiced black-fruit flavors end with pepper, wood grain, burnt toast and other oaky notes. Drink from 2018–2031. fakeword ',
       'Pleasant red fruits awash in chocolaty oak craft an accessible, broadly open Syrah. Though it seems a bit generic, it is balanced and tasty; a good bottle for near-term drinking with simple grilled meats. fakeword ',
       "This medium-bodied Sauvignon Blanc adds a bit of creamy depth on the midpalate, while its aromas and flavors of fig and melon are on the ripe side for Marlborough. Despite that, it's still clean and refreshing, with a long, citrus-pith finish. fakeword ",
       ...,
       'This is a tight, tangy wine that has been rounded out b

In [ ]:
# here we do the text pre-processing very fast
corpus=[]
for i in range (0,len(wine_subset)):
# remove punctuation
  wine_descriptions = re.sub('[^a-zA-Z0-9 ]','',wine_subset["description"][i]) #,joined_reviews
# remove stopwords
  wine_descriptions = remove_stopwords(wine_descriptions.lower())
# next we can stem or lemmatize
  wine_descriptions = [lemma.lemmatize(word) for word in wine_descriptions.split()]
# porter stemmer
#wine_descriptions = [stemmer.stem(word) for word in wine_descriptions.split()]
  wine_descriptions = " ".join(wine_descriptions)
  documentsp = wine_descriptions.split('fakeword')
  documentsp = documentsp[:-1]
  corpus.append(wine_descriptions)

In [ ]:
countVec = CountVectorizer()
X_raw = countVec.fit_transform(corpus)
X = X_raw.toarray()

In [ ]:
wine_subset.description[2]

"This medium-bodied Sauvignon Blanc adds a bit of creamy depth on the midpalate, while its aromas and flavors of fig and melon are on the ripe side for Marlborough. Despite that, it's still clean and refreshing, with a long, citrus-pith finish."

In [ ]:
documentsp[2]

IndexError: list index out of range

## Question 1

For all the remaining questions, we use the winemagdata130kv2 dataset, and we randomly subset 10000 reviews with a random state = 1693. We preprocess the customer reviews in the sense of Natural Language Processing (with all the pre-processing steps explained in class and by using the Count Vectorizer transform). We label "Superior" those wines with more than 90 points (in the target variable). The other wines whose rating was not more than 90 points, we label as "Inferior".We subset the data into Train, and the Test sets random state remains 1693) such that the Test set is about 30%, and we train a logistic regression classifier. When this model is applied to the Test set, we see that the number of Superior wine correctly classified as Superior is

In [ ]:
pts = wine_subset.points
y = pts.copy().values
y[pts>=91] = 1
y[(pts<91)] = 0

countVec = CountVectorizer()
X_raw = countVec.fit_transform(corpus)
X = X_raw.toarray()

xtrain, xtest, ytrain, ytest = tts(X,y,test_size=0.3,random_state=1693)

# Please complete your code to answer this question ...

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cls = LogisticRegression(random_state=1693, solver='lbfgs')
cls.fit(xtrain,ytrain)
ypred = cls.predict(xtest)
cm = confusion_matrix(ytest, ypred)
pd.DataFrame(cm, columns=['inf', 'sup'], index =['inf', 'sup'])

,inf,sup
inf,2053,180
sup,334,433


## Question 2

Given the data prepared for Question 1), we train a support vector classifier with a radially basis fucntion kernel, a gamma value of 0.03 and a C value of 50. When this model is applied to the Test set, we see that the number of Inferior wine misclassified as Superior is

In [ ]:
vectorizer = TfidfVectorizer()


# Please complete your code to answer this question ...

In [ ]:
X_raw = vectorizer.fit_transform(corpus)
X = X_raw.toarray()

In [ ]:
xtrain, xtest, ytrain, ytest = tts(X,y,test_size=0.3,random_state=1693)
model = SVC(kernel='rbf',gamma=0.03,C=50)
model.fit(xtrain,ytrain)

SVC(C=50, gamma=0.03)

In [ ]:
cm=confusion_matrix(ytest,model.predict(xtest))
pd.DataFrame(cm, columns=['inf', 'sup'], index =['inf', 'sup'])

,inf,sup
inf,2019,214
sup,349,418


## Question 3

For the support vector machine model with radial kernel, we want to find the best values of the gamma and C hyperparameters that maximize the potential for predicting correctly the brand of Superior wines. For this you will use 5-fold Stratified cross-validated area under the receiver operating characteristic curve ('auc') as a metric for evaluation, and the algorithm of simulated annealing presented in class.

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
# Simulated Annealing
def get_candidate_solution_nd(current_solution, bounds,temp):
    return [np.clip(xi + np.random.normal() * (bounds[i][1] - bounds[i][0]) * temp,bounds[i][0],bounds[i][1]) for i, xi in enumerate(current_solution)]

# Simulated Annealing algorithm for n dimensions with different bounds
def simulated_annealing_nd(objective, bounds, max_iterations, initial_temp, cooling_rate, dimensions):
    # Initialize with a random solution within the specific bounds for each dimension
    current_solution = [np.random.uniform(bounds[i][0], bounds[i][1]) for i in range(dimensions)]
    current_solution_cost = objective(current_solution)
    best_solution = current_solution[:]
    best_solution_cost = current_solution_cost

    # Current temperature
    temp = initial_temp

    for iteration in range(max_iterations):
        # Get a candidate solution respecting individual dimension bounds
        candidate_solution = get_candidate_solution_nd(current_solution, bounds,temp)
        candidate_solution_cost = objective(candidate_solution)

        # Check if the candidate solution is better
        if candidate_solution_cost < current_solution_cost:
            current_solution, current_solution_cost = candidate_solution, candidate_solution_cost
            if candidate_solution_cost < best_solution_cost:
                best_solution, best_solution_cost = candidate_solution, candidate_solution_cost
        else:
            # Accept worse solutions with a certain probability
            if np.random.uniform(0, 1) < np.exp((current_solution_cost - candidate_solution_cost) / temp):
                current_solution, current_solution_cost = candidate_solution, candidate_solution_cost

        # Decrease the temperature
        temp *= cooling_rate

        # Optional: Output some information
        if (iteration+1) % 10 == 0:
            print(f"Iteration {iteration+1}, Temperature: {temp:.2f}, Current Solution: {current_solution}, Current Cost: {current_solution_cost}")

    return best_solution, best_solution_cost

In [ ]:
# Based on the SA algorithm defined already above, please complete your code to answer this question ...
bounds = [(0.01, 2), (45, 55)]
dimensions = 2

def objective(solution):
    gamma, C = solution
    print(gamma, C)
    model = SVC(kernel='rbf',gamma=gamma,C=C)
    auc_scores= cross_val_score(model, X, y, cv=5, scoring='roc_auc')
    average_auc = np.mean(auc_scores)
    print(average_auc)
    return -average_auc

In [ ]:
best_solution, best_cost = simulated_annealing_nd(objective=objective, bounds=bounds, max_iterations=10, initial_temp=10, cooling_rate=0.95,
dimensions=dimensions)

1.2631591839657932 50.88831728616672
0.8656650165143762
2.0 55.0
0.862842558542589
2.0 55.0
0.862842558542589
2.0 45.0
0.862842558542589
0.01 55.0
0.8514873245341619
0.01 55.0
0.8514873245341619
2.0 45.0
0.862842558542589
2.0 45.0
0.862842558542589
2.0 45.0
0.862842558542589
2.0 45.0
0.862842558542589
0.01 45.0
0.8537108541644479
Iteration 10, Temperature: 5.99, Current Solution: [0.01, 45.0], Current Cost: -0.8537108541644479
0.01 45.0


## Question 4

For all the remaining questions, we use the winemagdata130kv2 dataset, and we randomly subset 10000 reviews with a random state = 1693. We preprocess the customer reviews in the sense of Natural Language Processing (with all the pre-processing steps explained in class and by using the TF-IDF transform). We label as Excellent those wines with at least 92 points (in the target variable), Good those between 87 and 91, and OK those below 87. We subset the data into Train, and the Test sets random state remains 1693) such that the Test set is about 25%, and we train a neural network classifier that has 300 neurons in the first layer and 200 neurons in the second layer, both layers having Parametric Rectified Linear Unit activations. Train the network using a batch size of 25, 50 epochs and the Adaptive momentum gradient descent with a learning rate of 0.001. Please apply this model to the Test set and show the resultant confusion matrix.




---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, PReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

prep1 = TfidfVectorizer()
X_raw = prep1.fit_transform(corpus)
X = X_raw.toarray()

pts = wine_subset.points
y = pts.copy().values
y[pts >= 92] = 2  # Excellent
y[(pts >= 87) & (pts <= 91)] = 1  # Good
y[pts < 87] = 0  # OK

Xtrain,Xtest,ytrain,ytest = tts(X,y,random_state=1693,test_size=0.25)

# Please complete your code to answer this question ...

In [ ]:
model = Sequential()
model.add(Dense(300, input_dim=12730)) #, activation='PReLU')
model.add(PReLU())
model.add(Dense(200)) #, activation='PReLU'))
model.add(PReLU())
model.add(Dense(3, activation='softmax'))
optimizer = Adam(learning_rate=.001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.fit(Xtrain, ytrain, epochs=50, batch_size=25) # , validation_data=(testX, testY) , verbose=0

Epoch 1/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 50s 146ms/step - accuracy: 0.5992 - loss: 0.8644
Epoch 2/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 27s 89ms/step - accuracy: 0.8636 - loss: 0.3595
Epoch 3/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 43s 96ms/step - accuracy: 0.9613 - loss: 0.1238
Epoch 4/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 39s 89ms/step - accuracy: 0.9926 - loss: 0.0283
Epoch 5/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 27s 88ms/step - accuracy: 0.9995 - loss: 0.0042
Epoch 6/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 26s 85ms/step - accuracy: 0.9994 - loss: 0.0026
Epoch 7/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 28s 93ms/step - accuracy: 1.0000 - loss: 4.2984e-04
Epoch 8/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 26s 88ms/step - accuracy: 1.0000 - loss: 2.3560e-04
Epoch 9/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 26s 88ms/step - accuracy: 1.0000 - loss: 1.6741e-04
Epoch 10/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 40s 86ms/step - accuracy: 1.0000 - loss: 1.2056e-04
Epoch 11/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - accuracy: 1.0000 - loss: 8.7383e-05
Epoch 12/

In [ ]:
y_pred = np.argmax(model.predict(Xtest), axis=1)

79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step


In [ ]:
cm = confusion_matrix(ytest,y_pred)
pd.DataFrame(cm, columns=['inf', 'okay', 'sup'], index =['inf', 'okay', 'sup'])

,inf,okay,sup
inf,415,237,2
okay,252,1019,158
sup,5,226,186


## Question 5:


Design your own convolutional neural network that will train on the MNIST data set and it will achieve at least 99% accuracy on the Test. The dataset can be directly downloaded from the library, using:

```python
from keras.datasets import mnist
(trainX, trainy), (testX, testy) = mnist.load_data()
```


In [ ]:
import tensorflow as tf

# example of loading the mnist dataset
from sklearn.model_selection import StratifiedKFold
#from sklearn.preprocessing import OneHotEncoder
# here we get close to the original format of MNIST this is 28x28 pixels for each image
from keras.datasets import mnist
from keras.models import Sequential # most of the time we design a "sequence" of layers
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
#from __future__ import print_function
import numpy as np
import keras

from tensorflow.keras.utils import to_categorical # this does the same as OneHotEncoder
# we import a lot of network design specific functions that we may need
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import LeakyReLU, ReLU
from keras.regularizers import l1, l2, l1_l2
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization


# load dataset - this is a different version of MNIST compared to sklearn
(trainX, trainy), (testX, testy) = mnist.load_data()


# Please complete your code to answer this question ...

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
def preprocess(x):
    return x.reshape(-1,28,28,1,order="F")/255.0
train_image = preprocess(trainX)
test_image = preprocess(testX)
train_y = to_categorical(trainy)
test_y = to_categorical(testy)
test_image=tf.convert_to_tensor(test_image)
train_image=tf.convert_to_tensor(train_image)
test_y=tf.convert_to_tensor(test_y)
train_y=tf.convert_to_tensor(train_y)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),input_shape=(28, 28, 1)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) #
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(10, activation='softmax'))
opt = Adam(learning_rate=0.002)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
model.fit(train_image, train_y, epochs=5, batch_size=40, validation_data=(test_image, test_y), verbose=1)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 71s 46ms/step - accuracy: 0.9019 - loss: 0.3042 - val_accuracy: 0.9881 - val_loss: 0.0377
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 64s 43ms/step - accuracy: 0.9840 - loss: 0.0519 - val_accuracy: 0.9870 - val_loss: 0.0379
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 66s 44ms/step - accuracy: 0.9863 - loss: 0.0428 - val_accuracy: 0.9902 - val_loss: 0.0287
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 79s 42ms/step - accuracy: 0.9896 - loss: 0.0337 - val_accuracy: 0.9895 - val_loss: 0.0327
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 83s 43ms/step - accuracy: 0.9910 - loss: 0.0288 - val_accuracy: 0.9905 - val_loss: 0.0290
